In [7]:
import numpy as np
from deap import base, creator, tools
import matplotlib
from epyt import epanet
from matplotlib.pyplot import plot
import random

In [118]:
p_c = 0.7
p_m = 0.1
dimension = 2
GroupAmount = 100
Encode_length = 14
FUNC_MIN=0.01

In [10]:
class Gen:
    def __init__(self):
        self.gen=np.zeros(Encode_length * dimension,dtype=bool)
        self.Eval=0

In [63]:
group = []
temp = []
p = []
N=0
temp_group=[]   
for i in range(groupAmount):
    group.append(Gen())
    temp.append(Gen())
    p.append(0)
    temp_group.append(Gen())

In [12]:
def cmp(a, b):
    return a.Eval>b.Eval

In [87]:
def decode(g, i):
    k = 1
    x = 0
    j = 0 + Encode_length * (i + 1) - 1
    while j >= i * Encode_length:
        if g.gen[j]:
            x += k
        k *= 2
        j -= 1
    return x * 100.0 / 16383

In [88]:
def init(g):
    j = 0
    while j < dimension:
        x = random.randint(0,10000)
        i = Encode_length * (j + 1) - 1
        while i >= j * Encode_length:
            g.gen[i] = x % 2
            x = x // 2
            i -= 1
        j += 1

In [18]:
def eval_func(g):
    return 0

In [19]:
def calculate_for_group():
    F = 0
    i = 0
    while i < GroupAmount:
        group[i].Eval = 1/(eval_func(group[i])+0.001)
        F += group[i].Eval
        i += 1
    
    p[0] = group[0].Eval / F
    i = 1
    while i < GroupAmount:
        p[i] = group[i].Eval / F + p[i-1]
        i += 1

In [56]:
def roulette_wheel():
    x = random.randint(0,10000)/10000.0
    left = 0
    right = GroupAmount - 1
    mid = (left+right) // 2 + 1
    if x <= p[0]:
        return 0
    elif x >= p[right-1]:
        return right
    while x > p[mid] or x <=p[mid-1]:
        if x == p[mid-1]:
            return left
        elif x > p[mid]:
            left = mid
            mid = (left + right)//2
        else:
            right = mid
            mid = (left + right)//2
    return mid - 1

In [22]:
def pick_for_temp():
    for i in range(GroupAmount):
        temp[i] = group[roulette_wheel()]

In [23]:
def single_point_crossover(index1, index2):
    x = random.randint(0,dimension * Encode_length)
    p_x = random.randint(0,999) / 1000.0
    if p_x < p_c:
        for i in range(dimension * Encode_length - x):
            t = temp[index1].gen[i+x]
            temp[index1].gen[i+x] = temp[index2].gen[i+x]
            temp[index2].gen[i+x] = t

In [58]:
def breed():
    for i in range(groupAmount//2):
        single_point_crossover(i*2,i*2+1)

In [60]:
def single_point_mutation():
    P_M = int((p_m+0.0001) * 1000)
    for i in range(GroupAmount):
        for j in range(dimension * Encode_length):
            x = random.randint(0,999)
            if x < P_M:
                temp[i].gen[j] = !temp[i].gen[j]

In [27]:
def calculate_for_temp():
    for i in range(GroupAmount):
        temp[i].Eval = 1/(eval_func(temp[i])+0.001)

In [67]:
def pick_for_group():
    group1 = sorted(group,key=lambda x:x.Eval)
    temp1 = sorted(temp,key=lambda x:x.Eval)
    i = 0
    j = 0
    for k in range(GroupAmount):
        if cmp(group[i],temp[j]):
            temp_group[k] = group1[i]
            i += 1
        else:
            temp_group[k] = temp1[j]
            j += 1
    for i in range(GroupAmount):
        group[i] = temp_group[i]

In [133]:
def Continue(N):
    if N>=1000:
        return False
    elif eval_func(group[0]) <= FUNC_MIN:
        return False
    else:
        return True

In [35]:
G=epanet('D:\epanet_1_example.inp')

# Plot network topology
#G.plot()

# Plot nodes IDs
#G.plot(nodesID=True)

# Plot links IDs
#G.plot(linksID=True)

# Plot node indices
#G.plot(nodesindex=True)

# Plot link indices
#G.plot(linksindex=True)

# Highlight specific links
#d.plot(highlightlink='10')

# Lists all available functions and properties
print(dir(G))

# Retrieve some examples for the function
help(G.getNodeElevations)

# Retrieve Link diameters
diameters = G.getLinkDiameter()
print(diameters)

# Retrieve Node elevations
elevations = G.getNodeElevations()
print(elevations)

# Link diameter for links 2 & 10
diameters = G.getLinkDiameter([1, 2])
print(diameters)



# Update the link 10 diameter from 100 to 90
G.setLinkDiameter(1, 400)
# Retrieve the diameter of link 10
n_diameter = G.getLinkDiameter(1)
print(n_diameter)

# Solve hydraulics in library
# H = G.getComputedHydraulicTimeSeries()
# Solve quality dynamics in library
# Q = G.getComputedQualityTimeSeries()
# Solve all dynamics in library, create a binary file to store the computed values

pmcs = G.getComputedHydraulicTimeSeries().Pressure
print(f"Epoch=  ",pmcs )
qmcs = G.getComputedHydraulicTimeSeries().Flow
print(f"Epoch=  ",qmcs )


roughness0=G.getLinkRoughnessCoeff([2,4])
print(roughness0)
head0=G.getNodePressure([4])
print(head0)

index_pipe=2
coff=88
G.setLinkRoughnessCoeff(index_pipe,coff)
coff1=G.getLinkRoughnessCoeff(index_pipe)
print(coff1)

EPANET version 20200 loaded (EPyT version 1.0.6).
Input File epanet_1_example.inp loaded successfully.

['BinTempfile', 'CMDCODE', 'ControlLevelValues', 'ControlLinkIndex', 'ControlNodeIndex', 'ControlRulesCount', 'ControlSettings', 'ControlTypes', 'ControlTypesIndex', 'Controls', 'CurveCount', 'CurveIndex', 'CurvesInfo', 'DEMANDMODEL', 'InputFile', 'Iterations', 'LOGOP', 'LibEPANET', 'LibEPANETpath', 'LinkBulkReactionCoeff', 'LinkCount', 'LinkDiameter', 'LinkFlowUnits', 'LinkIndex', 'LinkInitialSetting', 'LinkInitialStatus', 'LinkLength', 'LinkMinorLossCoeff', 'LinkNameID', 'LinkPipeCount', 'LinkPipeIndex', 'LinkPipeNameID', 'LinkPumpCount', 'LinkPumpHeadCurveIndex', 'LinkPumpIndex', 'LinkPumpNameID', 'LinkPumpPatternIndex', 'LinkPumpPatternNameID', 'LinkPumpPower', 'LinkPumpType', 'LinkPumpTypeCode', 'LinkRoughnessCoeff', 'LinkType', 'LinkTypeIndex', 'LinkValveCount', 'LinkValveIndex', 'LinkValveNameID', 'LinkWallReactionCoeff', 'NodeBaseDemands', 'NodeCoordinates', 'NodeCount', 'Nod

In [155]:
def eval_func(x):
    x1 = decode(x,0)
    x2 = decode(x,1)
    if x1 > 100 or x2 > 100:
        return 1000000

    index_pipe2 = 2
    index_pipe4 = 4
    G.setLinkRoughnessCoeff(index_pipe2,x1)
    G.setLinkRoughnessCoeff(index_pipe4,x2)

    pmcs = G.getComputedHydraulicTimeSeries().Pressure
    qmqs = G.getComputedHydraulicTimeSeries().Flow
    fitness=(G.getNodePressure([4])-3.0)
    return abs(fitness)

In [159]:
for i in range(GroupAmount):
    init(group[i])

In [153]:
def work(N):
    print("第",N,"代开始迭代")
    print(group[0].Eval,"  ",group[-1].Eval)
    calculate_for_group()
    pick_for_temp()
    breed()
    single_point_mutation()
    calculate_for_temp()
    print(group[0].Eval,"  ",group[-1].Eval)
    pick_for_group()
    print(group[0].Eval,"  ",group[-1].Eval)

In [156]:
N=0
while(Continue(N)):
    N+=1
    work(N)

第 1 代开始迭代
[0.29683286]    [0.29683286] 


[0.29683286]    [0.29683286] 


第 2 代开始迭代
[0.29683286]    [0.29683286] 


[0.29683286]    [0.29683286] 


第 3 代开始迭代


KeyboardInterrupt: 

In [162]:
group[0].gen

array([False,  True,  True,  True,  True,  True, False,  True, False,
        True,  True,  True,  True,  True, False,  True, False,  True,
        True, False,  True,  True,  True, False,  True, False, False,
       False,  True,  True])

TabError: inconsistent use of tabs and spaces in indentation (3228564415.py, line 159)